In [1]:
import torch
import math
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from torch.optim import Adam

import lightning as L
from torch.utils.data import TensorDataset, DataLoader

In [2]:
df = pd.read_csv('../cleaned/usdrub_new.csv')
columns = ['close']
df = df[columns]

In [3]:
training_length = math.floor(len(df) * 0.7)
train_df, test_df = df.iloc[:training_length], df.iloc[training_length:]

In [4]:
from CurrencyDataset import CurrencyDataset
train_data = CurrencyDataset(train_df,  seq_length = 5)
test_data = CurrencyDataset(test_df,  seq_length = 5)

In [97]:
train_dl = DataLoader(train_data, batch_size=8)
test_dl = DataLoader(test_data, batch_size=8)

In [98]:
print('Training Set:')
for sample in train_dl:  
    print(sample[0].size())
    print(sample[1].size())
    break

Training Set:
torch.Size([8, 5, 1])
torch.Size([8, 1])


In [162]:
class LSTM(L.LightningModule):
  def __init__(self):
    super().__init__()

    self.lstm = nn.LSTM(input_size=1, hidden_size=1, batch_first=True)
    self.linear = nn.Linear(in_features=1, out_features=1)

    self.example_input_array = torch.Tensor(8, 5, 1)

  def forward(self, input):
    lstm_output = self.lstm(input)
    
    # [batch_size, 1]
    last_unrolled_values = lstm_output[0][:, -1, :]
    prediction = self.linear(last_unrolled_values)
    return prediction
  
  def configure_optimizers(self):
    return Adam(self.parameters(), lr= 0.01)
  
  def training_step(self, batch, batch_idx):
    input = batch[0]
    label = batch[1]
    # [batch_size, 1]
    output = self.forward(input)
    loss = (output - label).pow(2).sum()

    self.log("train_loss", loss)
    return loss
  
  def test_step(self, batch, batch_idx):
    input = batch[0]
    label = batch[1]
    # [batch_size, 1]
    output = self.forward(input)
    loss = (output - label).pow(2).sum()

    self.log("test_loss", loss)
    return loss

In [141]:
class LightningLSTM(L.LightningModule):
  def __init__(self, lstm, learning_rate):
    super().__init__()
    self.lstm = lstm
    self.learning_rate = learning_rate

  def configure_optimizers(self):
    return Adam(self.parameters(), lr= self.learning_rate)
  
  def training_step(self, batch, batch_idx):
    input = batch[0]
    label = batch[1]
    # [batch_size, 1]
    output = self.lstm(input)
    loss = (output - label).pow(2).sum()

    self.log("train_loss", loss)
    return loss
  
  def test_step(self, batch, batch_idx):
    input = batch[0]
    label = batch[1]
    # [batch_size, 1]
    output = self.lstm(input)
    loss = (output - label).pow(2).sum()

    self.log("test_loss", loss)
    return loss

In [163]:
model = LSTM()

In [164]:
trainer = L.Trainer(max_epochs = 50, log_every_n_steps=2, profiler="simple")

GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [165]:
trainer.fit(model, train_dataloaders=train_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type   | Params | In sizes  | Out sizes                          
------------------------------------------------------------------------------------
0 | lstm   | LSTM   | 16     | [8, 5, 1] | [[8, 5, 1], [[1, 8, 1], [1, 8, 1]]]
1 | linear | Linear | 2      | [8, 1]    | [8, 1]                             
------------------------------------------------------------------------------------
18        Trainable params
0         Non-trainable params
18        Total params
0.000     Total estimated model params size (MB)
c:\Users\igor\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Epoch 49: 100%|██████████| 273/273 [00:01<00:00, 215.76it/s, v_num=13]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 273/273 [00:01<00:00, 214.74it/s, v_num=13]


FIT Profiler Report

---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                        	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                                                         	|  -              	|  492082         	|

In [134]:
trainer.test(model, dataloaders = test_dl)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\igor\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=19` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 117/117 [00:00<00:00, 583.29it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_loss         │      2404.7880859375      │
└───────────────────────────┴───────────────────────────┘

[{'test_loss': 2404.7880859375}]

In [131]:
# training some more
path_to_best_checkpoint = trainer.checkpoint_callback.best_model_path

In [132]:
trainer = L.Trainer(max_epochs=100)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [133]:
trainer.fit(model, train_dataloaders=train_dl, ckpt_path = path_to_best_checkpoint)

Restoring states from the checkpoint path at c:\Users\igor\Documents\GitHub\financial_tools\currency\lightning\lightning_logs\version_10\checkpoints\epoch=29-step=8190.ckpt
c:\Users\igor\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:361: The dirpath has changed from 'c:\\Users\\igor\\Documents\\GitHub\\financial_tools\\currency\\lightning\\lightning_logs\\version_10\\checkpoints' to 'c:\\Users\\igor\\Documents\\GitHub\\financial_tools\\currency\\lightning\\lightning_logs\\version_11\\checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type | Params
------------------------------
0 | lstm | LSTM | 18    
------------------------------
18        Trainable params
0         Non-trainable params
18        Total params
0.000     Total estimated model param

Epoch 99: 100%|██████████| 273/273 [00:01<00:00, 259.55it/s, v_num=11]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 273/273 [00:01<00:00, 258.57it/s, v_num=11]


In [77]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 18632), started 16:06:35 ago. (Use '!kill 18632' to kill it.)

In [158]:
from lightning.pytorch.utilities.model_summary import ModelSummary
summary = ModelSummary(model, max_depth=-1)
print(summary)

  | Name   | Type   | Params | In sizes  | Out sizes                          
------------------------------------------------------------------------------------
0 | lstm   | LSTM   | 16     | [8, 5, 1] | [[8, 5, 1], [[1, 8, 1], [1, 8, 1]]]
1 | linear | Linear | 2      | [8, 1]    | [8, 1]                             
------------------------------------------------------------------------------------
18        Trainable params
0         Non-trainable params
18        Total params
0.000     Total estimated model params size (MB)


In [29]:
# Tests
input = next(iter(train_dl))

In [30]:
input[1].size()

torch.Size([8, 1])

In [31]:
lstm = nn.LSTM(input_size=1, hidden_size=1, batch_first=True)

In [32]:
lstm_output = lstm(input[0])

In [33]:
lstm_output[0].size()

torch.Size([8, 5, 1])

In [41]:
last_unrolled_values = lstm_output[0][:, -1]

In [42]:
last_unrolled_values.size()

torch.Size([8, 1])

In [43]:
linear = nn.Linear(in_features=1, out_features=1)

In [23]:
linear_output = linear(last_unrolled_values)

In [25]:
linear_output.size()

torch.Size([8, 1])